In [ ]:
from dotenv import load_dotenv
_ = load_dotenv("../.env")

In [ ]:
import sys
sys.path.append("..")

In [ ]:
import json
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
from langchain_anthropic import ChatAnthropic

local_llm = "qwen2.5:7b"
# llm = ChatOllama(model=local_llm, temperature=0)
# llm = ChatOllama(model="qwen2.5:7b", temperature=0)
llm = ChatAnthropic(model="claude-3-5-sonnet-20241022", temperature=0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0, format="json")

In [ ]:
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma

In [ ]:
embedder = OllamaEmbeddings(model="nomic-embed-text")
db = Chroma(
        persist_directory="../data/chroma_layers", 
        embedding_function=embedder
)

In [ ]:
rag_prompt = """You are a World Resources Institute (WRI) assistant specializing in dataset recommendations.

Instructions:
1. Use the following context to inform your response:
{context}

2. User Question:
{question}

3. Response Format to be a valid JSON with list of datasets in the following format:
    {{
        "datasets": [
            {{
                "dataset": The slug of the dataset,
                "explanation": A two-line explanation of why this dataset is relevant to the user's problem
            }},
            ...
        ]
    }}
"""

In [ ]:
retriever = db.as_retriever(k=4)

In [ ]:
question = "I am interested in biodiversity conservation in Argentina"
docs = retriever.invoke(question)

In [ ]:
print(docs[0].page_content)

In [ ]:
def make_context(docs):
    fmt_docs = []
    for doc in docs:
        dataset = doc.metadata['dataset']
        content = (
            f"Dataset: {dataset}\n{doc.page_content}"
        )
        fmt_docs.append(content)
    
    # Join all formatted documents with double newlines
    return "\n\n".join(fmt_docs)

In [ ]:
docs_txt = make_context(docs)

In [ ]:
rag_prompt_fmt = rag_prompt.format(context=docs_txt, question=question)

In [ ]:
print(rag_prompt_fmt)

In [ ]:
generation = llm.invoke([HumanMessage(content=rag_prompt_fmt)])

In [ ]:
json.loads(generation.content)["datasets"]

In [ ]:
import operator
from typing_extensions import TypedDict
from typing import List, Annotated
from IPython.display import Image, display, Markdown
from langgraph.graph import START, MessagesState, StateGraph, END, add_messages

In [ ]:
class GraphState(TypedDict):
    question: str  # User question
    generation: str  # LLM generation
    loop_step: Annotated[int, operator.add]
    documents: List[str]  # List of retrieved documents

In [ ]:
def retrieve(state):
    print("---RETRIEVE---")
    question = state["question"]
    documents = retriever.invoke(question)
    return {"documents": documents}

def generate(state):
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    loop_step = state.get("loop_step", 0)

    # RAG generation
    docs_txt = make_context(documents)
    rag_prompt_fmt = rag_prompt.format(context=docs_txt, question=question)
    generation = llm.invoke([HumanMessage(content=rag_prompt_fmt)])
    datasets = json.loads(generation.content)["datasets"]
    for dataset in datasets:
        dataset["uri"] = f"https://data-api.globalforestwatch.org/dataset/{dataset['dataset']}"
        dataset["tilelayer"] = f"https://tiles.globalforestwatch.org/{dataset['dataset']}/latest/dynamic/{{z}}/{{x}}/{{y}}.png"

    return {"generation": datasets, "loop_step": loop_step + 1}

In [ ]:
wf = StateGraph(GraphState)

wf.add_node("retrieve", retrieve)
wf.add_node("generate", generate)

wf.add_edge(START, "retrieve")
wf.add_edge("retrieve", "generate")
wf.add_edge("generate", END)

graph = wf.compile()

In [ ]:
display(Image(graph.get_graph(xray=False).draw_mermaid_png()))

In [ ]:
result = graph.invoke({"question": "I am interested in tree cover loss over amazon"})

In [ ]:
result["generation"]

In [ ]:
response = json.loads(result["generation"].content)

In [ ]:
response